# Big Data w biznesie

## Drugi notebook rozwiązania

In [ ]:
from IPython.core.display import HTML

def _set_css_style(css_file_path):
   """
   Read the custom CSS file and load it into Jupyter.
   Pass the file path to the CSS file.
   """

   styles = open(css_file_path, "r").read()
   s = '<style>%s</style>' % styles
   return HTML(s)
_set_css_style("../custom.css")

#### Zaimportuj biblioteki numpy i pandas.

In [ ]:
import numpy as np
import pandas as pd

#### Załaduj plik csv.
***
Do zmiennej `df` załaduj plik `cars.csv`.

In [ ]:
df = pd.read_csv("./cars.csv")

#### Wyświetl pierwsze 5 linijek obiektu ''df''.

In [ ]:
df.head(5)

#### Sprawdź, które kolumny zamienić na typ 'category'.
***
Kolumny, w których wartości przyjmują tylko kilka różnych parametrów, warto zamienić typ na 'category'.

In [ ]:
df_unique = df.describe(include='all').loc['unique', :]

In [ ]:
list_of_category_columns = df_unique.loc[df_unique < 20].index.to_list()

#### Napisz funkcję, która nadaje kolumnom typ 'category'.
***
Funkcja jako argumenty będzie przyjmować obiekt Dataframe oraz listę z nazwami kolumn, którym chcielibyśmy zmienić typ na `category`.

In [ ]:
list_of_columns = ['transmission', 'color', 'engine_fuel', 'engine_type', 'body_type', 'state', 'drivetrain', 'location_region']

In [ ]:
def proper_columns_types(df, list_of_columns):
    for column in list_of_columns:
        df[column] = df[column].astype('category')
    return df

In [ ]:
df = proper_columns_types(df, list_of_columns)
df.info()

#### Wczytaj plik i od razu nadaj odpowiednim kolumnom typ 'category'.

In [ ]:
df = pd.read_csv("./cars.csv", dtype = {column:'category' for column in list_of_columns})
df.info()

#### Napisz funkcję, która printuje brakujące dane.
***
Funkcja ta jak argument ma przyjmować obiekt typu Dataframe i printować na ekran liczbę brakujących wartości dla każdej kolumny w podanym DataFrame.

In [ ]:
df_bad = df.apply(lambda x: x.isnull().value_counts()).loc[True,:]

In [ ]:
list_of_missing = df_bad.loc[df_bad > 1].index.tolist()

In [ ]:
def print_missing(df):
    for column in df: print(df.isnull()[column].value_counts())

#### Napisz funkcję, która zamienia losowe dane na 'np.nan'.
***
W związku z tym, że chcemy nauczyć się pracować z brakującymi danymi, najpierw musimy takie stworzyć.

Nasza funkcja jako argumenty będzie przyjmować obiekt Dataframe oraz słownik. Kluczem w słowniku będą nazwy kolumn, a wartościami będą liczby błędów, które chcemy wprowadzić (`{'odometer_value': 7}` oznacza, że w kolumnie `'odometer_value'` chcemy zamienić 7 losowych wierszy na typ `'np.nan'`)

Podpowiedź:

W ten sposób możemy zmienić wartość na ` 'value'` w losowym wierszu w kolumnie `'column'`.

`df.loc[df.sample().index, column] = 'value'`

In [ ]:
error_dict = {'odometer_value': 7, 'duration_listed': 10, 'color': 6, 'engine_type': 2, 'price_usd': 4}

In [ ]:
def introduce_nan(df, error_dict):
    for column, value in error_dict.items():
        df_sample = df.sample(n = value)
        df.loc[df_sample.index, column] = np.nan
    return df

#### Wprowadź błędy do danych.
***
Użyj napisanej powyżej funkcji do wprowadzenia brakujących wartości do obiektu `'df_err'`.

Sprawdź, czy udało Ci się wprowadzić te błędy za pomocą funkcji do printowania brakujących wartości.

In [ ]:
df_error = df.copy()

In [ ]:
df_error = introduce_nan(df_error, error_dict)
print_missing(df_error)

#### Zamień dane typu 'np.nan' na dane liczbowe.
***
W kolumnach `'odometer_value'` i `'duration_listed'` zamień brakujące wartości na średnią wartość całej kolumny.

W kolumnach `'color':` i `'engine_type'`: zamień brakujące wartości na najczęściej występujący element w całej kolumnie.

W kolumnie `'price_usd'` usuń brakujące wartości z naszego obiektu Dataframe.

In [ ]:
df_err = df_error.copy()

In [ ]:
avg_odometer_value = df_err['odometer_value'].astype("float").mean(axis=0)
print("Average of odometer :", avg_odometer_value)
df_err["odometer_value"] = df_err["odometer_value"].replace(np.nan, avg_odometer_value)

In [ ]:
avg_duration_listed = df_err['duration_listed'].astype("float").mean(axis=0)
print("Average of duration listed:", avg_duration_listed)
df_err['duration_listed'] = df_err['duration_listed'].replace(np.nan, avg_duration_listed)

In [ ]:
max_color_value = df_err['color'].value_counts().idxmax()
print("Max value of color:", max_color_value)
df_err['color'] = df_err['color'].replace(np.nan, max_color_value)

In [ ]:
max_engine_type = df_err['engine_type'].value_counts().idxmax()
print("Max value of engine_type:", max_engine_type)
df_err['engine_type'] = df_err['engine_type'].replace(np.nan, max_engine_type)

In [ ]:
df_err = df_err.dropna(subset=['price_usd'], axis=0)
df_err = df_err.reset_index(drop=True)

#### Napisz funkcję, która zamienia dane typu 'np.nan' na dane liczbowe.
***
Nasza funkcja jako argumenty będzie przyjmować obiekt Dataframe oraz słownik. Kluczem w słowniku będą nazwy kolumn, a wartościami będą wartości `'average'`,`'max'` albo `'drop'` (w zależności co chcemy zrobić z brakującymi danymi).


In [ ]:
replace_dict = {'odometer_value': 'average', 'duration_listed': 'average', 'color': 'max', 'engine_type': 'max', 'price_usd': 'drop'}

In [ ]:
def replace_column(df, replace_dict):
    df=df.replace('?',np.nan)
    for column, value in replace_dict.items():
        if value == 'average':
            avg = df[column].astype("float").mean(axis=0)
            df[column] = df[column].fillna(value = avg)
        elif value == 'max':
            max_value = df[column].value_counts().idxmax()
            df[column] = df[column].fillna(value = max_value)
        elif value == 'drop':
            df = df.dropna(subset=[column], axis=0)
            df = df.reset_index(drop=True)
        else:
            print("Value cannot be replaced!")
    return df

#### Wprowadź błędy do danych.
***
Użyj napisanej powyżej funkcji do zastąpienia lub usunięcia brakujących wartości z obiektu `'df_err'`.

Sprawdź, czy udało Ci się to zrobić za pomocą funkcji do printowania brakujących wartości.

In [ ]:
df_err = replace_column(df_err, replace_dict)
print_missing(df_err)

#### Napisz zapytanie.
***
Znajdź wszystkie samochody marki Mercedes-Benz wyprodukowane po 2010 roku.

In [ ]:
max_year = 2010
df.query('manufacturer_name == "Mercedes-Benz" and year_produced > @max_year') ## use query

#### Policz rok produkcji podniesiony do kwadratu.
***
W obiekcie `'df_sec'` stwórz nowe kolumny `'year_produced_squared_1'` i `'year_produced_squared_2'` w których znajdzie się rok produkcji podniesiony do kwadratu.

Zrób to na dwa sposoby:
- działanie bezpośrednio na kolumnie
- działanie przy pomocy funkcji `.apply(lambda x: x)`

In [ ]:
df_sec = df.copy()

In [ ]:
df_sec['year_produced_squared'] = df_sec['year_produced'] ** 2

In [ ]:
df_sec['year_produced_squared_1'] = df_sec['year_produced'].apply(lambda x: x**2) ## worse way to do it, but lambda can be anything

#### Oceń czy samochód można uznać za nowy.
***
W obiekcie `'df_sec'` stwórz nową kolumnę `'is_new'`. Wartość powinna wynosić `True` dla wszystkich samochodów wyprodukowanych po 2016 i z przebiegiem mniejszym niż 100 000 km (oba warunki spełnione jednocześnie). W pozostałych przypadkach wartość powinna wynosić `'False'`.

In [ ]:
df_sec['is_new'] = np.logical_and(df_sec['year_produced'] > 2016, df_sec['odometer_value'] < 100000)

#### Policz średnią liczbę przebytych kilometrów na rok.
***
W obiekcie `'df_sec'` stwórz nową kolumnę `'km_per_year'`. Wartość powinna wynosić liczbę przebytych kilometrów podzieloną na lata użytkowania (zakładamy, że dane pochodzą z 2019 roku).

In [ ]:
df_sec['km_per_year'] = df_sec['odometer_value'] / (2019 - df_sec['year_produced']) ## use vectorized functions

#### Napisz funkcję, która liczy przebyte kilometry w ciągu roku.
***
Nasza funkcja jako argumenty będzie przyjmować obiekt Dataframe oraz rok, z którego pochodzą dane.
Funkcja będzie zwracać obiekt typu Dataframe, który w kolumnie `'kpy'` ma policzoną tę wartość.

In [ ]:
## use functions for repeated actions
def calculate_kpy(df, year):
    df['kpy'] = df['odometer_value'] / (year - df['year_produced'])
    return df

#### Jeszcze raz policz średnią liczbę przebytych kilometrów na rok.
***
Na obiekcie `'df_sec'` użyj napisanej powyżej funkcji (zakładamy, że dane pochodzą z 2019 roku).

In [ ]:
df_sec = calculate_kpy(df_sec, 2019)

#### Ustandaryzuj dane.
***
W obiekcie `'df_sec'` stwórz nowe kolumny `'odometer_value_min_max_norm'` i `'odometer_value_norm_dist_norm'` w których ustandaryzowana (znormalizowana) wartość z kolumny `'odometer_value'`.

Zrób to na dwa sposoby:
- standaryzacja z użyciem wartości minimalnych i maksymalnych
- standaryzacja z użyciem wartości średniej i odchylenia standardowego

In [ ]:
df_sec['odometer_value_min_max_norm'] = (df_sec['odometer_value'] - df_sec['odometer_value'].min())/(df_sec['odometer_value'].max() - df_sec['odometer_value'].min())

In [ ]:
df_sec['odometer_value_norm_dist_norm'] = (df_sec['odometer_value'] - df_sec['odometer_value'].mean()) / df_sec['odometer_value'].std()

#### Zmień wszystkie dane w kolumnie na małe litery.
***
W obiekcie `'df_sec'` stwórz nową kolumnę `'model_name_lower'`, która będzie zawierać wartości z kolumny `'model_name'`, ale ze wszystkimi wielkimi literami zmienionymi na małe.

In [ ]:
df_sec['model_name_lower'] = df_sec['model_name'].str.lower() ## string methods
df_sec

#### Zmień nazwy kolumn.
***
W obiekcie `'df_sec'` zmień nazwy kolumn `'manufacturer_name'` oraz `'model_name'` na odpowiednio `'Manufacturer_Name'` i `'Model_Name'`.

In [ ]:
changed_names_dict = {'manufacturer_name':'Manufacturer_Name', 'model_name':'Model_Name'}

In [ ]:
## rename by using proper function
df_sec = df_sec.rename(columns=changed_names_dict)
df_sec

#### Policz średnią wartość odczytu z licznika w zależności od marki i modelu samochodu.
***
Pogrupuj dane po kolumnach `'Manufacturer_Name'` oraz `'Model_Name'` i policz dla nich średnią z kolumny `'odometer_value'`.

In [ ]:
df_odo = df_sec.groupby(['Manufacturer_Name', 'Model_Name'])[['odometer_value']].mean()
df_odo

#### Policz średnią wartość pojemności silnika w zależności od typu skrzyni biegów i liczbę wystąpień danych typów skrzyni biegów.
***
Pogrupuj dane po kolumnie `'transmission'` i policz dla nich średnią z kolumny `'engine_capacity'` oraz policz liczbę obserwacji w każdej grupie.

In [ ]:
## aggregating instead of iterating
df_engine = df_sec.groupby(['transmission'])['engine_capacity'].agg(['mean', 'count'])
df_engine